# Library imports

In [1]:
from scipy.stats import percentileofscore as score
from scipy import stats
import pandas as pd
import numpy as np 
import xlsxwriter 
import requests
import math

# Importing Our List of Stocks and getting API Token


In [2]:
pd.set_option('display.max_rows', None)
stocks = pd.read_csv('sp_500_stocks.csv')
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2312022185980,
 'week52high': 185.68,
 'week52low': 130.29,
 'week52highSplitAdjustOnly': 186.17,
 'week52lowSplitAdjustOnly': 132.84,
 'week52change': -0.2067558637439746,
 'sharesOutstanding': 16691892188,
 'float': 0,
 'avg10Volume': 68663030,
 'avg30Volume': 73516460,
 'day200MovingAvg': 153.6,
 'day50MovingAvg': 152.1,
 'employees': 153060,
 'ttmEPS': 6.41,
 'ttmDividendRate': 0.9378362858878821,
 'dividendYield': 0.00665453174818612,
 'nextDividendDate': '',
 'exDividendDate': '2022-10-31',
 'nextEarningsDate': '2023-01-19',
 'peRatio': 23.526877992624716,
 'beta': 1.29712858623072,
 'maxChangePercent': 55.87176681718114,
 'year5ChangePercent': 2.6162183602328444,
 'year2ChangePercent': 0.18233654544473465,
 'year1ChangePercent': -0.181816609513403,
 'ytdChangePercent': -0.2011098295032746,
 'month6ChangePercent': -0.0003292569741094245,
 'month3ChangePercent': -0.09594304101621233,
 'month1ChangePercent': 0.05523017859683062,
 'day30Cha

# Executing a batch API Call & Building our Dataframe

In this section, we will filter for stocks with the lowest percentiles on the following metrics:

    1. Price-to-earnings ratio (Equivalent in portuguese: P/L - Preço sobre lucro)
    2. Price-to-book ratio (Equivalent in portuguese: P/VP - Preço sobre Valor Patrimonial)
    3. Price-to-sales ratio (Equivalent in portuguese: P/S - Preço sobre vendas
    4. Enterprise Value divided by earnings before interest, taxes, depreciation, and amortization (EV/EBITDA)
    5. Enterprise Value divided by gross profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data.

In [3]:
rv_columns = [
    'Ticker',
    'companyName',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score',
    'Aumento de preco 1 ANO',
    'Aumento de preco 2 ANOS',
    'Aumento de preco 5 ANOS',
    'dividend yield'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)
rv_dataframe

,Ticker,companyName,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score,Aumento de preco 1 ANO,Aumento de preco 2 ANOS,Aumento de preco 5 ANOS,dividend yield


In [4]:
def chunks(lst,n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i : i + n]

symbol_groups = list(chunks(stocks['Ticker'],100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    
    symbol_strings.append(','.join(symbol_groups[i]))
    #print (symbol_strings[i])
    
    
for symbol_string in symbol_strings[:6]:      
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string} &types=quote,price,stats,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        
        
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']

        # Enterprise value divided by Gross Profit (EV/GP)
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except:
            ev_to_gross_profit = np.NaN
        
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['companyName'],
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',            
                'N/A',
                data[symbol]['stats']['year1ChangePercent'],
                data[symbol]['stats']['year2ChangePercent'],
                data[symbol]['stats']['year5ChangePercent'],
                data[symbol]['stats']['dividendYield']
        ],
        index = rv_columns), ignore_index = True)
        
rv_dataframe
     

,Ticker,companyName,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score,Aumento de preco 1 ANO,Aumento de preco 2 ANOS,Aumento de preco 5 ANOS,dividend yield
0,AAP,Advance Auto Parts Inc,147.100,N/A,17.77,N/A,3.2,N/A,0.832,N/A,9.869930,N/A,2.026940,N/A,N/A,-0.373447,-0.037482,0.542507,0.037915
1,AAPL,Apple Inc,146.540,N/A,24.03,N/A,45.75,N/A,5.75,N/A,18.304707,N/A,13.838928,N/A,N/A,-0.182435,0.188907,2.593689,0.006443
2,ABBV,Abbvie Inc,165.490,N/A,22.65,N/A,18.32,N/A,5.11,N/A,12.739388,N/A,8.924603,N/A,N/A,0.368988,0.647838,1.155524,0.034546
3,ABC,Amerisource Bergen Corp.,168.410,N/A,21.1,N/A,-161.1,N/A,0.1465,N/A,10.713404,N/A,4.467890,N/A,N/A,0.386268,0.69716,1.120146,0.011703
4,ABMD,Abiomed Inc.,386.290,N/A,66.3,N/A,11.36,N/A,16.2,N/A,57.966924,N/A,18.738120,N/A,N/A,0.229979,0.447118,1.009773,0.0
5,ABT,Abbott Laboratories,108.350,N/A,25.11,N/A,5.37,N/A,4.34,N/A,16.639570,N/A,7.582390,N/A,N/A,-0.181768,0.045562,1.174813,0.017414
6,ACN,Accenture plc - Class A,299.260,N/A,26.96,N/A,8.99,N/A,3.21,N/A,15.757732,N/A,9.262700,N/A,N/A,-0.215356,0.204509,1.099881,0.014406
7,ADBE,Adobe Inc,336.310,N/A,32.96,N/A,10.96,N/A,9.02,N/A,24.459726,N/A,9.833221,N/A,N/A,-0.480958,-0.326105,0.936626,0.0
8,ADI,Analog Devices Inc.,169.740,N/A,32.62,N/A,2.44,N/A,7.33,N/A,20.005392,N/A,12.023020,N/A,N/A,-0.065803,0.240876,1.199021,0.018476
9,ADM,Archer Daniels Midland Co.,96.070,N/A,12.76,N/A,2.2,N/A,0.5207,N/A,9.667033,N/A,7.771210,N/A,N/A,0.479642,0.94882,1.560446,0.017823


# Dealing with Missing Data in Our DataFrame
Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using.

You can use pandas' isnull method to identify missing data.

In [5]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)].index

Int64Index([ 26,  38,  69, 115, 118, 132, 161, 182, 186, 189, 200, 249, 320,
            321, 322, 343, 355, 437, 447, 488],
           dtype='int64')

Dealing with missing data is an important topic in data science.

There are 2 main approaches:

1. Drop missing data from the data set (pandas's dropna method is useful here)
2. Replace missing data with a new value (pandas' fillna method is useful here)

In [6]:
rv_dataframe_filled = rv_dataframe
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio', 'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP', 'Aumento de preco 1 ANO', 'Aumento de preco 2 ANOS', 'Aumento de preco 5 ANOS']:
    rv_dataframe_filled[column].fillna(rv_dataframe[column].mean(), inplace = True)
rv_dataframe_filled

,Ticker,companyName,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score,Aumento de preco 1 ANO,Aumento de preco 2 ANOS,Aumento de preco 5 ANOS,dividend yield
0,AAP,Advance Auto Parts Inc,147.100,N/A,17.770000,N/A,3.200000,N/A,0.832000,N/A,9.869930,N/A,2.026940,N/A,N/A,-0.373447,-0.037482,0.542507,0.037915
1,AAPL,Apple Inc,146.540,N/A,24.030000,N/A,45.750000,N/A,5.750000,N/A,18.304707,N/A,13.838928,N/A,N/A,-0.182435,0.188907,2.593689,0.006443
2,ABBV,Abbvie Inc,165.490,N/A,22.650000,N/A,18.320000,N/A,5.110000,N/A,12.739388,N/A,8.924603,N/A,N/A,0.368988,0.647838,1.155524,0.034546
3,ABC,Amerisource Bergen Corp.,168.410,N/A,21.100000,N/A,-161.100000,N/A,0.146500,N/A,10.713404,N/A,4.467890,N/A,N/A,0.386268,0.697160,1.120146,0.011703
4,ABMD,Abiomed Inc.,386.290,N/A,66.300000,N/A,11.360000,N/A,16.200000,N/A,57.966924,N/A,18.738120,N/A,N/A,0.229979,0.447118,1.009773,0.0
5,ABT,Abbott Laboratories,108.350,N/A,25.110000,N/A,5.370000,N/A,4.340000,N/A,16.639570,N/A,7.582390,N/A,N/A,-0.181768,0.045562,1.174813,0.017414
6,ACN,Accenture plc - Class A,299.260,N/A,26.960000,N/A,8.990000,N/A,3.210000,N/A,15.757732,N/A,9.262700,N/A,N/A,-0.215356,0.204509,1.099881,0.014406
7,ADBE,Adobe Inc,336.310,N/A,32.960000,N/A,10.960000,N/A,9.020000,N/A,24.459726,N/A,9.833221,N/A,N/A,-0.480958,-0.326105,0.936626,0.0
8,ADI,Analog Devices Inc.,169.740,N/A,32.620000,N/A,2.440000,N/A,7.330000,N/A,20.005392,N/A,12.023020,N/A,N/A,-0.065803,0.240876,1.199021,0.018476
9,ADM,Archer Daniels Midland Co.,96.070,N/A,12.760000,N/A,2.200000,N/A,0.520700,N/A,9.667033,N/A,7.771210,N/A,N/A,0.479642,0.948820,1.560446,0.017823


Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [7]:
rv_dataframe_filled[rv_dataframe_filled.isnull().any(axis=1)]
rv_dataframe_filled


,Ticker,companyName,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score,Aumento de preco 1 ANO,Aumento de preco 2 ANOS,Aumento de preco 5 ANOS,dividend yield
0,AAP,Advance Auto Parts Inc,147.100,N/A,17.770000,N/A,3.200000,N/A,0.832000,N/A,9.869930,N/A,2.026940,N/A,N/A,-0.373447,-0.037482,0.542507,0.037915
1,AAPL,Apple Inc,146.540,N/A,24.030000,N/A,45.750000,N/A,5.750000,N/A,18.304707,N/A,13.838928,N/A,N/A,-0.182435,0.188907,2.593689,0.006443
2,ABBV,Abbvie Inc,165.490,N/A,22.650000,N/A,18.320000,N/A,5.110000,N/A,12.739388,N/A,8.924603,N/A,N/A,0.368988,0.647838,1.155524,0.034546
3,ABC,Amerisource Bergen Corp.,168.410,N/A,21.100000,N/A,-161.100000,N/A,0.146500,N/A,10.713404,N/A,4.467890,N/A,N/A,0.386268,0.697160,1.120146,0.011703
4,ABMD,Abiomed Inc.,386.290,N/A,66.300000,N/A,11.360000,N/A,16.200000,N/A,57.966924,N/A,18.738120,N/A,N/A,0.229979,0.447118,1.009773,0.0
5,ABT,Abbott Laboratories,108.350,N/A,25.110000,N/A,5.370000,N/A,4.340000,N/A,16.639570,N/A,7.582390,N/A,N/A,-0.181768,0.045562,1.174813,0.017414
6,ACN,Accenture plc - Class A,299.260,N/A,26.960000,N/A,8.990000,N/A,3.210000,N/A,15.757732,N/A,9.262700,N/A,N/A,-0.215356,0.204509,1.099881,0.014406
7,ADBE,Adobe Inc,336.310,N/A,32.960000,N/A,10.960000,N/A,9.020000,N/A,24.459726,N/A,9.833221,N/A,N/A,-0.480958,-0.326105,0.936626,0.0
8,ADI,Analog Devices Inc.,169.740,N/A,32.620000,N/A,2.440000,N/A,7.330000,N/A,20.005392,N/A,12.023020,N/A,N/A,-0.065803,0.240876,1.199021,0.018476
9,ADM,Archer Daniels Midland Co.,96.070,N/A,12.760000,N/A,2.200000,N/A,0.520700,N/A,9.667033,N/A,7.771210,N/A,N/A,0.479642,0.948820,1.560446,0.017823


# Dealing with Price-to-Earnings Ratio < 0

An negative PeRatio indicates that the company has a negative profit, it means, a loss. 
In this approach, I'll remove companies that had a loss.

In [8]:
for row in rv_dataframe_filled.index:
    if (rv_dataframe_filled.loc[row,'Price-to-Earnings Ratio'] < 0):
        rv_dataframe_filled.drop(row)      
                
rv_dataframe_filled
rv_dataframe = rv_dataframe_filled

# Calculating Value Percentiles
Metrics:

 1. price to earnings ratio
 2. price to book ratio
 3. price to sales ratio
 4. EV/EBITDA
 5. EV/GP

In [9]:
from scipy.stats import percentileofscore as score

metrics = {
    'Price-to-Earnings Ratio': 'PE Percentile',
    'Price-to-Book Ratio' : 'PB Percentile',
    'Price-to-Sales Ratio': 'PS Percentile',
    'EV/EBITDA': 'EV/EBITDA Percentile',
    'EV/GP' : 'EV/GP Percentile',
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])

rv_dataframe

,Ticker,companyName,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score,Aumento de preco 1 ANO,Aumento de preco 2 ANOS,Aumento de preco 5 ANOS,dividend yield
0,AAP,Advance Auto Parts Inc,147.100,N/A,17.770000,38.554217,3.200000,54.919679,0.832000,12.851406,9.869930,30.120482,2.026940,11.445783,N/A,-0.373447,-0.037482,0.542507,0.037915
1,AAPL,Apple Inc,146.540,N/A,24.030000,54.819277,45.750000,98.192771,5.750000,82.931727,18.304707,73.493976,13.838928,90.562249,N/A,-0.182435,0.188907,2.593689,0.006443
2,ABBV,Abbvie Inc,165.490,N/A,22.650000,52.610442,18.320000,94.37751,5.110000,78.915663,12.739388,42.369478,8.924603,69.678715,N/A,0.368988,0.647838,1.155524,0.034546
3,ABC,Amerisource Bergen Corp.,168.410,N/A,21.100000,47.991968,-161.100000,0.803213,0.146500,0.60241,10.713404,32.730924,4.467890,32.53012,N/A,0.386268,0.697160,1.120146,0.011703
4,ABMD,Abiomed Inc.,386.290,N/A,66.300000,94.176707,11.360000,89.759036,16.200000,99.096386,57.966924,98.995984,18.738120,97.389558,N/A,0.229979,0.447118,1.009773,0.0
5,ABT,Abbott Laboratories,108.350,N/A,25.110000,59.437751,5.370000,71.787149,4.340000,74.096386,16.639570,67.068273,7.582390,60.843373,N/A,-0.181768,0.045562,1.174813,0.017414
6,ACN,Accenture plc - Class A,299.260,N/A,26.960000,63.855422,8.990000,84.538153,3.210000,58.433735,15.757732,62.248996,9.262700,72.088353,N/A,-0.215356,0.204509,1.099881,0.014406
7,ADBE,Adobe Inc,336.310,N/A,32.960000,75.301205,10.960000,89.156627,9.020000,91.566265,24.459726,87.951807,9.833221,75.903614,N/A,-0.480958,-0.326105,0.936626,0.0
8,ADI,Analog Devices Inc.,169.740,N/A,32.620000,75.100402,2.440000,41.064257,7.330000,87.349398,20.005392,79.518072,12.023020,84.738956,N/A,-0.065803,0.240876,1.199021,0.018476
9,ADM,Archer Daniels Midland Co.,96.070,N/A,12.760000,25.502008,2.200000,35.140562,0.520700,6.425703,9.667033,29.116466,7.771210,62.851406,N/A,0.479642,0.948820,1.560446,0.017823


# Calculating RV Score

We'll now calculate our RV Score (Robust Value), which is the value score that we'll use to filter for stocks in this invest strategy. 

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section. 

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module. 

In [10]:
from statistics import mean

for row in rv_dataframe.index:
    pe_ratio = rv_dataframe.loc[row, 'PE Percentile']
    pb_ratio = rv_dataframe.loc[row, 'PB Percentile']
    ps_ratio = rv_dataframe.loc[row, 'PS Percentile']
    ev_to_ebitda = rv_dataframe.loc[row, 'EV/EBITDA Percentile']
    ev_to_gp = rv_dataframe.loc[row, 'EV/GP Percentile']
    rv_dataframe.loc[row, 'RV Score'] = round(mean([pe_ratio, pb_ratio, ps_ratio, ev_to_ebitda, ev_to_gp]))
    
rv_dataframe[['Ticker', 'PE Percentile', 'PB Percentile', 'PS Percentile', 'EV/EBITDA Percentile', 'EV/GP Percentile', 'RV Score']]

,Ticker,PE Percentile,PB Percentile,PS Percentile,EV/EBITDA Percentile,EV/GP Percentile,RV Score
0,AAP,38.554217,54.919679,12.851406,30.120482,11.445783,30
1,AAPL,54.819277,98.192771,82.931727,73.493976,90.562249,80
2,ABBV,52.610442,94.37751,78.915663,42.369478,69.678715,68
3,ABC,47.991968,0.803213,0.60241,32.730924,32.53012,23
4,ABMD,94.176707,89.759036,99.096386,98.995984,97.389558,96
5,ABT,59.437751,71.787149,74.096386,67.068273,60.843373,67
6,ACN,63.855422,84.538153,58.433735,62.248996,72.088353,68
7,ADBE,75.301205,89.156627,91.566265,87.951807,75.903614,84
8,ADI,75.100402,41.064257,87.349398,79.518072,84.738956,74
9,ADM,25.502008,35.140562,6.425703,29.116466,62.851406,32


# Selecting the 10th best value stocks

In [11]:
rv_dataframe.sort_values('RV Score', ascending=True, inplace=True)
rv_dataframe.reset_index(drop=True, inplace=True)
rv_dataframe.head(10)


,Ticker,companyName,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score,Aumento de preco 1 ANO,Aumento de preco 2 ANOS,Aumento de preco 5 ANOS,dividend yield
0,GM,General Motors Company,39.79,N/A,6.64,11.445783,0.8514,9.036145,0.3790,4.417671,1.938421,1.807229,1.507121,8.835341,7,-0.359001,-0.138177,-0.005855,0.004712
1,KSS,Kohl`s Corp.,26.49,N/A,6.30,10.64257,0.7310,8.032129,0.1606,0.803213,4.435168,8.032129,1.122756,5.421687,7,-0.470428,-0.306835,-0.372833,0.076661
2,AIG,American International Group Inc,65.56,N/A,3.71,5.62249,1.2700,14.35743,0.8020,12.449799,2.243230,2.008032,0.813346,2.811245,7,0.170041,0.675722,0.205105,0.02049
3,C,Citigroup Inc,45.12,N/A,6.19,10.441767,0.4850,7.228916,1.0008,16.064257,3.697314,4.618474,0.977570,3.815261,8,-0.256119,-0.190530,-0.309922,0.046913
4,COF,Capital One Financial Corp.,98.65,N/A,4.73,7.429719,0.7246,7.831325,1.0000,15.863454,3.374818,3.614458,1.139578,5.823293,8,-0.351640,0.032012,0.075427,0.025241
5,BA,Boeing Co.,181.56,N/A,-13.00,3.413655,-6.0500,6.2249,1.8200,32.028112,-28.321644,0.60241,-184.597574,0.200803,8,-0.135140,-0.233701,-0.347820,0.0
6,UNM,Unum Group,41.44,N/A,7.10,12.851406,0.9406,9.437751,0.6793,10.040161,3.889897,5.823293,0.688680,2.008032,8,0.735069,0.869935,-0.149491,0.03195
7,PVH,PVH Corp,74.35,N/A,5.64,9.236948,0.9395,9.236948,0.5420,7.028112,5.559467,11.445783,1.205346,6.626506,9,-0.294389,-0.225367,-0.465141,0.002127
8,SYF,Synchrony Financial,35.70,N/A,5.81,9.437751,1.3400,15.562249,0.9015,13.453815,3.426191,4.016064,1.046236,4.417671,9,-0.259741,0.136917,0.068070,0.025739
9,F,Ford Motor Co.,13.41,N/A,5.96,10.040161,1.2800,14.658635,0.3466,3.413655,3.838417,5.220884,1.692956,9.839357,9,-0.308146,0.454602,0.246800,0.038369


# Removing stocks that didn't give profit

In [34]:
rv_dataframe.shape


(498, 19)

In [35]:
for row in rv_dataframe.index:
    if ((rv_dataframe.loc[row,'Aumento de preco 5 ANOS'] and rv_dataframe.loc[row,'Aumento de preco 2 ANOS']) < 0):
        #print(rv_dataframe.loc[row,'companyName'])
        rv_dataframe.drop(row, inplace=True) 
        


In [36]:
rv_dataframe.shape

(357, 19)

# Final Dataframe

In [37]:
final_rv_dataframe = rv_dataframe[['Ticker','companyName','RV Score','Aumento de preco 1 ANO','Aumento de preco 2 ANOS','Aumento de preco 5 ANOS','dividend yield' ]]
final_rv_dataframe

,Ticker,companyName,RV Score,Aumento de preco 1 ANO,Aumento de preco 2 ANOS,Aumento de preco 5 ANOS,dividend yield
2,AIG,American International Group Inc,7,0.170041,0.675722,0.205105,0.02049
4,COF,Capital One Financial Corp.,8,-0.351640,0.032012,0.075427,0.025241
6,UNM,Unum Group,8,0.735069,0.869935,-0.149491,0.03195
8,SYF,Synchrony Financial,9,-0.259741,0.136917,0.068070,0.025739
9,F,Ford Motor Co.,9,-0.308146,0.454602,0.246800,0.038369
11,PHM,PulteGroup Inc,11,-0.199512,0.072076,0.374173,0.013946
12,NRG,NRG Energy Inc.,11,-0.132987,0.022307,0.289930,0.044328
14,CAH,"Cardinal Health, Inc.",12,0.680839,0.496087,0.593110,0.026035
15,DXC,DXC Technology Co,12,-0.144564,0.062094,-0.674574,0
16,VLO,Valero Energy Corp.,12,0.710938,1.161481,0.655305,0.035401


# Formatting our excel output

In [38]:
writer = pd.ExcelWriter('value_strategy3.xlsx', engine = 'xlsxwriter')
final_rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index=False)


In [39]:
background_color = '#0a0a23'
background_data_color = '#ffffff'
font_color = '#ffffff'
font_data_color = '#000000'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.00',
            'font_color': font_data_color,
            'bg_color': background_data_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [40]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['companyName', string_template],
                    'C': ['RV Score', float_template],
                    'D': ['Aumento de preco 1 ANO', float_template],
                    'E': ['Aumento de preco 2 ANOS', float_template],
                    'F': ['Aumento de preco 5 ANOS', float_template],
                    'G': ['dividend yield', float_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [41]:
writer.save()